<font color=gray>[https://www.kaggle.com/mantri7/imdb-movie-reviews-dataset] Es la calificación de un set de peliculas donde en la escala de 1 a 10 => 5>positiva o menor que 5 negativa, al final sera una clasificación binaria, en formato de texto, __toca predecir si la valoración que se esta dando es positiva o negativa, solo subministrandole el texto__. Tiene 25K para entrenar y 25k para test</font>

In [1]:
import tensorflow as tf
# Internet Movie Data Base
from tensorflow.keras.datasets import imdb

In [2]:
tf.__version__

'2.1.0'

### Pre-prosesado de Datos

<font color=gray>__Se le parasaran valoraciones de la misma longitud, es un requisito que todas tengan la misma longitud para empezar, en tal caso se añadiran palabras adicionales para que todas queden con la misma longitud___</font>

__number_of_words__
    * Vocavulario: Cuando importemos las valoraciones de las peliculas vamos a quedarnos con aquellas que tengan las palabras mas frecuentes, el objetivo es quedarno con las valoraciones que tienen sentido.
__max_long__
    * Si tenemos una valoración con 5 palabras, esta se va a representar como una valoración de 100 elementos donde los 5 primeros seran el contenido Original y los otros 95 una misma palabra repetida hasta completar 100.

In [3]:
# Configurar parámetros del dataset
number_of_words = 20000
max_len = 100

In [4]:
# Carga del data set de IMDB
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = number_of_words)

In [5]:
# Cortar Secuencias de texto de la misma longitud para que todas queden de 100 elementos
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen = max_len )

In [6]:
vocab_size = number_of_words
vocab_size

20000

###  Construir la Red Neuronal Recurrente

In [7]:
embed_size = 128

In [8]:
# Definir el modelo
model = tf.keras.Sequential()

Añadir la capa de embedding (incrustar, añadir, colocar). Toma los textos antes de ser subministrados a la red neuronal. __Esta capa__ crea un vector de palabras traduciendolo a una matriz de 0roz y 1nos, Esto se hace con la funcion de Embedding se utiliza para entrenar grandes matrizes o vectores y tranformarla en una gran matriz, __donde cada fila se corresponde con una de las valoraciones__ y las columnas son precizamente las palabras.

__Parametros__
    * vocab_size => 20 mil potenciales palabras
    * embed_size => 128 pesos para cada palabra que se este analizando.
    * input_shape => Se indica el tamaño de entrada (todas las valoraciones)

In [9]:
# Añadir la capa de embedding
model.add(tf.keras.layers.Embedding(vocab_size, embed_size, input_shape = (X_train.shape[1],)))

__Añadir la capa de LSTM__
    * unidades: 128
    * función de activación: tanh


<font color=red> https://en.wikipedia.org/wiki/Long_short-term_memory </font>

In [10]:
model.add(tf.keras.layers.LSTM(units = 128, activation = 'tanh'))

<font color=gray>__Dense__ capa totalmente contectada, que dara como respuesta una neurona que dara la valoración positiva o negativa y ademas hara uso de la funcion __sigmoid__ que me transformara el resultado anterior en una probabilidad entre [0,1]</font>

In [11]:
# Añadir la capa totalmente conectada de salida
model.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

### Compilar el modelo

__Parametros__
    * 'rmsprop' => la proporción del error cuadrado medio (es el recomendado para RNR)
    * 'binary_crossentropy' => pues la operación es binaria 
    * metrica =>  asociada es accuracy o es cierto o es falso 

In [12]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
# 20.000 columnas * 128 neuronas (pesos)
# LSTM filtrado con la tangente hyperbolica 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 128)          2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


### Entrenar el modelo

In [17]:
model.fit(X_train, y_train, epochs=3, batch_size=128)

Train on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 45s 2ms/sample - loss: 0.2335 - accuracy: 0.9108
Epoch 2/3
25000/25000 [==============================] - 50s 2ms/sample - loss: 0.1882 - accuracy: 0.9300
Epoch 3/3
25000/25000 [==============================] - 47s 2ms/sample - loss: 0.1572 - accuracy: 0.9436


In [15]:
test_loss, test_acurracy = model.evaluate(X_test, y_test)

25000/25000 [==============================] - 25s 1000us/sample - loss: 0.3949 - accuracy: 0.8478


In [16]:
print("Test accuracy: {}".format(test_acurracy))

Test accuracy: 0.8478400111198425
